# 毕业论文`GRAD`
## `CMAQ`模拟效果验证

---
*@author: Evan*\
*@date: 2023-12-19*

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import matplotlib.pyplot as plt
from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import sys
sys.path.append('../../src/')
from namelist import *
from mask import polygon_to_mask
import ModelEvalLib as me

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
years = [2019,2021,2022]
month = 'Jul'

simvar = 'PM25'
obsvar = 'PM2.5'

In [3]:
def plot_line_short(year,mean_obs,mean_sim,mb,r,rmse):
    fig = plt.figure(figsize=(18,6),dpi=300)
    ax = fig.subplots(1,1)

    xx = np.arange(0,np.size(mean_obs,0),1)
    ax.plot(xx,mean_sim,color='tab:red',label='SIM',linestyle='-', marker='.',markersize=10)
    ax.plot(xx,mean_obs,color='tab:blue',label='OBS',linestyle='--',marker='.',markersize=10)

    ax.set_xlim([0,np.size(mean_obs,0)-1])
    ax.set_xticks(xx[::24])
    STR = get_STR(year,month)
    END = get_END(year,month)
    ax.set_xticklabels(pd.date_range(STR,END,freq='D').strftime('%m-%d'))#,rotation=45)

    strdict = {'fontsize':16,'fontweight':'bold'}
    ax.set_xlabel('Date',fontdict=strdict)
    ax.set_ylabel('PM$_{2.5}$ ($\mu$g/m$^3$)',fontdict=strdict)
    ax.set_title('(c)',loc='left',fontdict=strdict)
    ax.legend(loc=2)

    # ===========
    # add metrics
    mb_str   = '{:.2f}'.format(mb)
    r_str    = '{:.2f}'.format(r)
    rmse_str = '{:.2f}'.format(rmse)

    ax.text(0.73,1.01,f'R = {r_str}',transform=ax.transAxes,fontdict=strdict,ha='right')
    ax.text(0.85,1.01,f'MB = {mb_str}',transform=ax.transAxes,fontdict=strdict,ha='right')
    ax.text(0.99,1.01,f'RMSE = {rmse_str}',transform=ax.transAxes,fontdict=strdict,ha='right')

    plt.savefig(f'D:/Academic/Project/GRAD/Seasonally/figures/模拟验证/{simvar}_{year}.png',dpi=300,bbox_inches='tight')
    plt.close()

In [4]:
def plot_line_long(year,mean_obs,mean_sim,mb,r,rmse):
    fig = plt.figure(figsize=(24,3),dpi=300)
    ax = fig.subplots(1,1)

    xx = np.arange(0,np.size(mean_obs,0),1)
    ax.plot(xx,mean_sim,color='tab:red',label='SIM',linestyle='-', marker='.',markersize=10)
    ax.plot(xx,mean_obs,color='tab:blue',label='OBS',linestyle='--',marker='.',markersize=10)

    ax.set_xlim([0,np.size(mean_obs,0)-1])
    ax.set_xticks(xx[::24])
    STR = get_STR(year,month)
    END = get_END(year,month)
    ax.set_xticklabels(pd.date_range(STR,END,freq='D').strftime('%m-%d'))#,rotation=45)

    strdict = {'fontsize':16,'fontweight':'bold'}
    ax.set_xlabel('Date',fontdict=strdict)
    ax.set_ylabel('PM$_{2.5}$ ($\mu$g/m$^3$)',fontdict=strdict)
    ax.set_title('(c)',loc='left',fontdict=strdict)
    ax.legend(loc=2)

    # ===========
    # add metrics
    mb_str   = '{:.2f}'.format(mb)
    r_str    = '{:.2f}'.format(r)
    rmse_str = '{:.2f}'.format(rmse)

    ax.text(0.73,1.01,f'R = {r_str}',transform=ax.transAxes,fontdict=strdict,ha='right')
    ax.text(0.85,1.01,f'MB = {mb_str}',transform=ax.transAxes,fontdict=strdict,ha='right')
    ax.text(0.99,1.01,f'RMSE = {rmse_str}',transform=ax.transAxes,fontdict=strdict,ha='right')

    plt.savefig(f'D:/Academic/Project/GRAD/Seasonally/figures/模拟验证/新-珠三角/{simvar}_{year}.png',dpi=300,bbox_inches='tight')
    plt.close()

In [5]:
for year in years:
    print(f'Year = {year}')

    # Simulation data
    ds = xr.open_dataset(datadir + f'processed/{month}_{year}/{month}_{year}_chem.nc')
    data_sim = ds[simvar][:,0,:,:]
    shp = gpd.read_file(shp_files['PRD_merge_adm'])
    lon = data_sim.longitude
    lat = data_sim.latitude
    mask    = polygon_to_mask(shp.geometry[0], lon, lat)
    mask_da = xr.DataArray(mask, dims=('y','x'))
    masked_sim  = data_sim.where(mask_da)
    mean_sim    = masked_sim.mean(dim=('x','y'),skipna=True)
    sim = mean_sim.resample(time='D').mean()
    
    # Observation data
    obspath = get_obspath(month)
    df = pd.read_excel(obspath + f'site_{obsvar}_{year}.xlsx',index_col=0)
    mean_obs = df.mean(axis=1,skipna=True)
    obs = mean_obs.resample('D').mean()
    
    # 检查sim与obs维度是否一致
    if sim.shape != obs.shape:
        raise ValueError('sim and obs have different dimensions!')
    
    # 使用逐小时值计算统计量，观测序列需对空值做填充
    inteobs = mean_obs.interpolate(method='linear')
    metrics = me.CalculateMetrics(inteobs,mean_sim)
    mb   = metrics.get_mb().values
    r    = metrics.get_r()
    rmse = metrics.get_rmse().values
    ioa  = metrics.get_ioa().values
    nmb  = metrics.get_nmb().values
    nme  = metrics.get_nme().values
    print('MB  : ',mb,'\nR   : ',r,'\nRMSE: ',rmse,'\nIOA : ',ioa,'\nNMB : ',nmb,'\nNME : ',nme)
    
    # plot_line_short(year,mean_obs,mean_sim,mb,r,rmse)
    plot_line_long(year,mean_obs,mean_sim,mb,r,rmse)
    print('-----------------')
    
    

Year = 2019
MB  :  -8.47016719321476 
R   :  0.8576013939941614 
RMSE:  9.975342276047305 
IOA :  0.5622938485955284 
NMB :  -52.8797792283027 
NME :  62.276680642939176
-----------------
Year = 2021
MB  :  -6.84830994457943 
R   :  0.561401840610709 
RMSE:  8.932443820212413 
IOA :  0.5519416688271277 
NMB :  -50.56982663682446 
NME :  65.95965122589911
-----------------
Year = 2022
MB  :  -6.228574517538189 
R   :  0.9020500475114209 
RMSE:  7.696744180186763 
IOA :  0.7092350256385518 
NMB :  -44.929691175666484 
NME :  55.520302133365185
-----------------
